In [1]:
import math
import numpy as np
import os
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pandas as pd
import random
# pd.set_option('display.max_rows', None)



In [2]:
def cal_mae_mse_rmse(path_pre,path_test):
    df_pre = pd.read_csv(path_pre, sep=' ',header=None).astype(float)  # 如果是逗号分隔的话
    df_pre.columns=['x_coord unit1 scale1', 'y_coord unit1 scale1', 'z_coord unit1 scale1','S2X4_POR unit1 scale1']
    df_test = pd.read_csv(path_test).astype(float)  # 如果是逗号分隔的话
    # print(df_test.shape,df_pre.shape)
    df_test['z_coord unit1 scale1']=df_test['z_coord unit1 scale1'].round(0)
    df_pre['z_coord unit1 scale1']=df_pre['z_coord unit1 scale1'].round(0)

    df_merge=pd.merge(df_test, df_pre, on=['x_coord unit1 scale1', 'y_coord unit1 scale1', 'z_coord unit1 scale1'], how='left')
    df_merge=df_merge.fillna(0)
    test_por=df_merge['S2X4_POR unit1 scale1_x']/100
    pre_por=df_merge["S2X4_POR unit1 scale1_y"]/100
    df_combined = pd.DataFrame({'test_por': test_por, 'pre_por': pre_por})

    return np.array([mean_absolute_error(test_por,pre_por),mean_squared_error(test_por,pre_por),sqrt(mean_squared_error(test_por,pre_por))]),df_combined


In [3]:
path_test_07="./test/known_points_test_0.3.csv"

IDW_path_pre_07_5="./result/res_IDW_zones_0.7_5_blur.txt"
IDW_error,df_IDW=cal_mae_mse_rmse(IDW_path_pre_07_5,path_test_07)
IDW_error

array([0.04930463, 0.00420477, 0.06484422])

In [4]:
path_test_08="./test/known_points_test_0.2.csv"

IDW_path_pre_08_5="./result/res_IDW_zones_0.8_5_blur.txt"
IDW_error,df_IDW=cal_mae_mse_rmse(IDW_path_pre_08_5,path_test_08)

RBF_path_pre_08_5="./result/res_RBF_zones_0.8_5_blur.txt"
RBF_error,df_RBF=cal_mae_mse_rmse(RBF_path_pre_08_5,path_test_08)

OK_path_pre_08_5="./result/res_OK_zones_0.8_5_blur.txt"
OK_error,df_OK=cal_mae_mse_rmse(OK_path_pre_08_5,path_test_08)

UK_path_pre_08_5="./result/res_UK_zones_0.8_5_blur.txt"
UK_error,df_UK=cal_mae_mse_rmse(UK_path_pre_08_5,path_test_08)

RFS_path_pre_08_5="./result/res_RFS_zones_0.8_5_blur.txt"
RFS_error,df_RFS=cal_mae_mse_rmse(RFS_path_pre_08_5,path_test_08)

RFR_path_pre_08_5="./result/res_RFR_zones_0.8_5_blur.txt"
RFR_error,df_RFR=cal_mae_mse_rmse(RFR_path_pre_08_5,path_test_08)

df_error_08 = pd.DataFrame()
df_error_08['80%']=['MAE','MSE','RMSE']
df_error_08['IDW'] =IDW_error
df_error_08['RBF'] =RBF_error
df_error_08['OK'] =OK_error
df_error_08['UK'] =UK_error
df_error_08['RFS'] =RFS_error
df_error_08['RFR'] =RFR_error
df_error_08


,80%,IDW,RBF,OK,UK,RFS,RFR
0,MAE,0.019210,0.023192,0.017859,0.018399,0.019298,0.010625
1,MSE,0.000655,0.000889,0.000562,0.000607,0.000671,0.000184
2,RMSE,0.025595,0.029814,0.023710,0.024636,0.025900,0.013562


In [6]:
def cal_UQ(path):
    #  读取文件内容并转换为list
    with open(path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    # 去除每行末尾的换行符并返回list
    lines = [line.strip() for line in lines]
    lines=[item for item in lines if item != '']
    float_data = [list(map(float, item.split())) for item in lines]
    numpy_array = np.array(float_data)
    mse_all=(numpy_array.reshape(-1))

    mean_value = np.sum(mse_all) / len(mse_all)/10
    print(path,np.sum(mse_all),len(mse_all),mean_value)
    len(mse_all)
    for v in mse_all:
        if v>400:
            print(v)
    return mean_value

UQ_IDW=cal_UQ("UQ_IDW.txt")
UQ_RBF=cal_UQ("UQ_RBF.txt")
UQ_OK=cal_UQ("UQ_OK.txt")
UQ_RFR=cal_UQ("UQ_RFR.txt")

UQ_IDW.txt 264.3381 47858 0.000552338376029086
UQ_RBF.txt 350.9237 46057 0.0007619334737390624
UQ_OK.txt 198.3906 50193 0.00039525551371705225
UQ_RFR.txt 187.23127823129212 50413 0.00037139483512445623
